In [1]:
from ocpmodels.trainers import EnergyTrainer
from ocpmodels.datasets import SinglePointLmdbDataset
from ocpmodels import models
from ocpmodels.common import logger
from ocpmodels.common.utils import setup_logging
setup_logging()

import numpy as np
import copy
import os

import torch
print(torch.__version__)
print(torch.version.cuda)

ERROR:root:Invalid setup for SCN. Either the e3nn library or Jd.pt is missing.


2.0.0
11.8


In [2]:
train_src = "ocp/data/is2re/10k/train/data.lmdb"
val_src = "ocp/data/is2re/all/val_id/data.lmdb"

dataset = [
    {
        "src": train_src,
        "normalize_labels": True,
        "target_mean": -1.525913953781128,
        "target_std": 2.279365062713623,
    },
    {"src": val_src}, # val data
    {"src": val_src}, # test data
]

dataset

[{'src': 'ocp/data/is2re/10k/train/data.lmdb',
  'normalize_labels': True,
  'target_mean': -1.525913953781128,
  'target_std': 2.279365062713623},
 {'src': 'ocp/data/is2re/all/val_id/data.lmdb'},
 {'src': 'ocp/data/is2re/all/val_id/data.lmdb'}]

In [3]:
task = {
  "dataset": "single_point_lmdb",
  "description": "Relaxed state energy prediction from initial structure.",
  "type": "regression",
  "metric": "mae",
  "labels": ["relaxed energy"],
}
# Model
model = {    
    'name': 'dimenetplusplus',
    "num_spherical": 7,
    "num_radial": 6,
    "num_blocks": 3,
    "hidden_channels": 256,
    "out_emb_channels": 192,
    "num_before_skip": 1,
    "num_after_skip": 2,
    "num_output_layers": 3,
    "cutoff": 6.0,
    "use_pbc": True,
    "regress_forces": False,
}
# Optimizer
optimizer = {
    'batch_size': 4,
    'eval_batch_size': 4,
    'num_workers': 4,
    'lr_initial': 1.e-4,
    'lr_gamma': 0.1,
    'lr_milestones': [115082, 230164, 345246],
    'warmup_steps': 57541,
    'warmup_factor': 0.2,
    'max_epochs': 20,
}

In [4]:
# Path to the pre-trained model file
model_path = 'dimenetpp_all.pt'

pretrained_energy_trainer = EnergyTrainer(
    task=task,
    model=model,
    dataset=dataset,
    optimizer=optimizer,
    identifier="IS2RE-val-example",
    run_dir="./", # directory to save results if is_debug=False. Prediction files are saved here so be careful not to override!
    is_debug=False, # if True, do not save checkpoint, logs, or results
    print_every=10,
    seed=0, # random seed to use
    logger="tensorboard", # logger of choice (tensorboard and wandb supported)
    local_rank=0,
    amp=True, # use PyTorch Automatic Mixed Precision (faster training and less memory usage)
)

pretrained_energy_trainer.load_checkpoint(checkpoint_path=model_path)

amp: true
cmd:
  checkpoint_dir: ./checkpoints/2023-07-06-13-40-16-IS2RE-val-example
  commit: null
  identifier: IS2RE-val-example
  logs_dir: ./logs/tensorboard/2023-07-06-13-40-16-IS2RE-val-example
  print_every: 10
  results_dir: ./results/2023-07-06-13-40-16-IS2RE-val-example
  seed: 0
  timestamp_id: 2023-07-06-13-40-16-IS2RE-val-example
dataset:
  normalize_labels: true
  src: ocp/data/is2re/10k/train/data.lmdb
  target_mean: -1.525913953781128
  target_std: 2.279365062713623
gpus: 1
logger: tensorboard
model: dimenetplusplus
model_attributes:
  cutoff: 6.0
  hidden_channels: 256
  num_after_skip: 2
  num_before_skip: 1
  num_blocks: 3
  num_output_layers: 3
  num_radial: 6
  num_spherical: 7
  out_emb_channels: 192
  regress_forces: false
  use_pbc: true
noddp: false
optim:
  batch_size: 4
  eval_batch_size: 4
  lr_gamma: 0.1
  lr_initial: 0.0001
  lr_milestones:
  - 115082
  - 230164
  - 345246
  max_epochs: 20
  num_workers: 4
  warmup_factor: 0.2
  warmup_steps: 57541
slurm:

In [5]:
# make predictions on the existing test_loader
predictions = pretrained_energy_trainer.predict(pretrained_energy_trainer.test_loader, results_file="is2re_results", disable_tqdm=False)

device 0: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6236/6236 [04:12<00:00, 24.68it/s]


In [6]:
predictions